<a href="https://www.kaggle.com/code/kisarak/real-esrgan-video-upscale?scriptVersionId=219612516" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## 1. Set Up Environment and Prepare ESRGAN

In [ ]:
%%capture --no-stderr
!pip install ffmpeg opencv-python opencv-python-headless basicsr facexlib gfpgan
!pip install -U git+https://github.com/xinntao/Real-ESRGAN.git


In [ ]:
# Download the REAL ESRGAN Repo on first run
# !git clone https://github.com/xinntao/Real-ESRGAN.git
# !wget -P Real-ESRGAN/experiments/pretrained_models https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth


In [ ]:
%%capture --no-stderr
# Requirements

import os
os.chdir("Real-ESRGAN")

# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# facexlib and gfpgan are for face enhancement
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

os.chdir("..")

In [ ]:
%%capture --no-stderr
!pip install torch==1.11.0 torchvision==0.12.0 --force-reinstall
!pip install "numpy<2.0" --force-reinstall


## 2. Get Video and Split Frames

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id={id}' -O '../video.mp4'


In [ ]:
import os

# Create a directory to save frames
os.makedirs('../frames', exist_ok=True)

# Extract frames from the video
!ffmpeg -hwaccel cuda -i '../video.mp4' -q:v 1 ../frames/frame%04d.png


## 3. Main Algorithm (Parallel)

### Split Folders

In [ ]:
import os
import shutil
from glob import glob

# Declare folder and frame range variables
start_frame = 1
end_frame = 9000

gpu0_folder = '../frames_gpu0'
gpu1_folder = '../frames_gpu1'

# Create directories for each GPU
os.makedirs(gpu0_folder, exist_ok=True)
os.makedirs(gpu1_folder, exist_ok=True)

# List all frames
frames = sorted(glob('../frames/*.png'))

# Process each frame and sort them into odd/even directories
for frame in frames:
    # Extract frame number from filename
    frame_number = int(os.path.basename(frame).replace('frame', '').replace('.png', ''))
    
    # Determine if frame is odd or even and move to the respective directory
    if start_frame <= frame_number <= end_frame:
        if frame_number % 2 == 1:  # Odd frame
            shutil.copy(frame, gpu0_folder)
        else:  # Even frame
            shutil.copy(frame, gpu1_folder)

### Process Frames

In [ ]:
import os
import subprocess
import threading
import time

# Set up the environment for GPU 0 and GPU 1
env_gpu0 = os.environ.copy()
env_gpu1 = os.environ.copy()

env_gpu0["CUDA_VISIBLE_DEVICES"] = "0"
env_gpu1["CUDA_VISIBLE_DEVICES"] = "1"

gpu0_upscaled_folder = gpu0_folder + '_upscaled'
# gpu1_upscaled_folder = gpu1_folder + '_upscaled'
gpu1_upscaled_folder = gpu0_upscaled_folder

# Output directories
os.makedirs(gpu0_upscaled_folder, exist_ok=True)
os.makedirs(gpu1_upscaled_folder, exist_ok=True)

# Run Real-ESRGAN on GPU 0
process_gpu0 = subprocess.Popen([
    "python", "-u", "Real-ESRGAN/inference_realesrgan.py",
    "-n", "RealESRGAN_x4plus",
    "-i", gpu0_folder,
    "-o", gpu0_upscaled_folder
], env=env_gpu0, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

# Run Real-ESRGAN on GPU 1
process_gpu1 = subprocess.Popen([
    "python", "-u", "Real-ESRGAN/inference_realesrgan.py",
    "-n", "RealESRGAN_x4plus",
    "-i", gpu1_folder,
    "-o", gpu1_upscaled_folder
], env=env_gpu1, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

start_time = time.time()

def get_time():
    t = time.time() - start_time
    hours, remainder = divmod(t, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours)}:{int(minutes):02}:{int(seconds):02}"

# Function to monitor progress
def monitor_progress(process, gpu_id):
    processed_frames = 0
    for line in iter(process.stdout.readline, ""):
        if processed_frames %10 == 0:
            print(f"{get_time()} GPU {gpu_id}: {line.strip()}")
        processed_frames += 1
    process.stdout.close()
    process.wait()
    print(f"{get_time()} GPU {gpu_id}: Completed processing.")

# Start monitoring in parallel
thread_gpu0 = threading.Thread(target=monitor_progress, args=(process_gpu0, 0))
thread_gpu1 = threading.Thread(target=monitor_progress, args=(process_gpu1, 1))

thread_gpu0.start()
thread_gpu1.start()

# Wait for both threads to finish
thread_gpu0.join()
thread_gpu1.join()


## 4. FFMPEG with GPU

In [ ]:
ffmpeg_command = [
    "ffmpeg",
    "-y",
    "-r", "29.97",
    "-start_number", f"{start_frame}",
    "-i", os.path.join(gpu0_upscaled_folder, "frame%04d_out.png"),
    "-vf", "scale=2160:3840",
    "-c:v", "h264_nvenc",
    "-cq", "18",
    "-preset", "slow",
    "-pix_fmt", "yuv420p",
    "../output_video_4k.mp4"
]

# Run the ffmpeg command
# try:
#     subprocess.run(ffmpeg_command, check=True, capture_output=True, text=True)
# except subprocess.CalledProcessError as e:
#     print(f"Error: {e.stderr}")

process = subprocess.Popen(ffmpeg_command, stderr=subprocess.PIPE, universal_newlines=True)

start_time = time.time()

# Parse and print progress
try:
    line_counter = 0
    for line in process.stderr:
        line = line.strip()
        if "frame=" in line or "time=" in line:  # Look for progress-related lines
            line_counter += 1
            if line_counter % 10 == 0:  # Print only every 10th line
                print(f"{get_time()} {line}")  # Print progress information
    process.wait()  # Wait for the process to finish
    if process.returncode != 0:
        raise subprocess.CalledProcessError(process.returncode, ffmpeg_command)
except subprocess.CalledProcessError as e:
    print(f"Error: {e}")

In [ ]:
from IPython.display import FileLink

video_path = "output_video_4k.mp4"
shutil.move('../' + video_path, video_path)

FileLink(video_path)

In [ ]:
shutil.move(video_path, '../' + video_path)


### FFMPEG with CPU

In [ ]:
import os
import shutil
import subprocess

# start_frame = 1

# Build the ffmpeg command
ffmpeg_command = [
    "ffmpeg",
    "-y",
    "-r", "29.97",
    "-start_number", f"{start_frame}",
    "-i", os.path.join(gpu0_upscaled_folder, "frame%04d_out.png"),
    "-vf", "scale=3840:2160",
    "-c:v", "libx264",
    "-crf", "16",
    "-preset", "slow",
    "-pix_fmt", "yuv420p",
    "output_video_4k.mp4"
]

# Run the ffmpeg command
# try:
#     subprocess.run(ffmpeg_command, check=True, capture_output=True, text=True)
# except subprocess.CalledProcessError as e:
#     print(f"Error: {e.stderr}")

process = subprocess.Popen(ffmpeg_command, stderr=subprocess.PIPE, universal_newlines=True)

start_time = time.time()

# Parse and print progress
try:
    line_counter = 0
    for line in process.stderr:
        line = line.strip()
        if "frame=" in line or "time=" in line:  # Look for progress-related lines
            line_counter += 1
            if line_counter % 10 == 0:  # Print only every 10th line
                print(f"{get_time()} {line}")  # Print progress information
    process.wait()  # Wait for the process to finish
    if process.returncode != 0:
        raise subprocess.CalledProcessError(process.returncode, ffmpeg_command)
except subprocess.CalledProcessError as e:
    print(f"Error: {e}")

In [ ]:
# Build the ffmpeg command
ffmpeg_command = [
    "ffmpeg",
    "-y",
    "-r", "29.97",
    "-start_number", f"{start_frame}",
    "-i", os.path.join(gpu0_upscaled_folder, "frame%04d_out.png"),
    "-vf", "crop=5120:2160:0:360",
    "-c:v", "libx264",
    "-crf", "16",
    "-preset", "slow",
    "-pix_fmt", "yuv420p",
    "output_video_5k2k.mp4"
]


# Run the ffmpeg command
# subprocess.run(ffmpeg_command, check=True)

process = subprocess.Popen(ffmpeg_command, stderr=subprocess.PIPE, universal_newlines=True)

start_time = time.time()

# Parse and print progress
try:
    line_counter = 0
    for line in process.stderr:
        line = line.strip()
        if "frame=" in line or "time=" in line:  # Look for progress-related lines
            line_counter += 1
            if line_counter % 10 == 0:  # Print only every 10th line
                print(f"{get_time()} {line}")  # Print progress information
    process.wait()  # Wait for the process to finish
    if process.returncode != 0:
        raise subprocess.CalledProcessError(process.returncode, ffmpeg_command)
except subprocess.CalledProcessError as e:
    print(f"Error: {e}")

In [ ]:
from IPython.display import Video, HTML, FileLink

video_path = "output_video_5k2k.mp4"

# # Display the video inline
# display(Video(video_path, embed=True, width=800))

# # Provide a download link
# download_link = f'<a href="{video_path}" download>Click here to download the video</a>'
# display(HTML(download_link))

FileLink(video_path)

In [ ]:
def delete_frames(folder):
    for file in os.listdir(folder):
        frame_number = int(os.path.basename(file).replace('frame', '').replace('_out.png', ''))
        if frame_number > 2400:
            file_path = os.path.join(folder, file)
            os.remove(file_path)
            print(f"Deleted: {file_path}")

# Delete frames in both folders
# delete_frames(gpu1_upscaled_folder)
# delete_frames(gpu0_upscaled_folder)

### Delete Folders to Free Space (For Iterations)

In [ ]:
!rm -rf ../frames_gpu1
!rm -rf ../frames_gpu0


In [ ]:
# !rm -rf ../frames_gpu1_upscaled
!rm -rf ../frames_gpu0_upscaled


In [ ]:
os.chdir('working')

In [ ]:
!rm -rf frames
!rm -f *.mkv


In [ ]:
# os.makedirs('../Irina', exist_ok=True)

# for file in sorted(os.listdir(gpu0_upscaled_folder)):
#     frame_number = int(os.path.basename(file).replace('frame', '').replace('_out.png', ''))
#     if 1950 < frame_number <= 2000 or True:
#         source_path = os.path.join(gpu0_upscaled_folder, file)
#         destination_path = os.path.join('../Irina', file)
#         shutil.copy(source_path, destination_path)
#         # print(frame_number)

In [ ]:
# shutil.make_archive('Irina', 'zip', '../Irina')

In [ ]:
ls

In [ ]:
ls ../

In [ ]:
!du -sh ../

In [ ]:
ls ../frames_gpu0_upscaled/

In [ ]:
# !rm -rf ../Irina
# !rm -rf Irina.zip

In [ ]:
# !rm -rf frames

In [ ]:
# os.chdir("..")

In [ ]:
# os.chdir("working")


In [ ]:
# !rm -f /kaggle/working/frames_z.zip
# !rm -rf frames


In [ ]:
# import shutil

# # Zip the folder
# shutil.make_archive('frames_z', 'zip', 'frames')